In [ ]:
import xarray as xr
import numpy as np
from scipy.special import lambertw

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from numba import vectorize
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
WBT_matrix = pd.read_csv('/home/581/cg8052/rhversion_WBT_matrix.csv', header=0, index_col=0)

In [ ]:

temperature = np.arange(25,60,0.1)
humidity =  np.arange(0.5, 100.5, 0.5)
xx_temp, yy_humidity = np.meshgrid(temperature, humidity)
xx = xx_temp
yy = yy_humidity

In [ ]:
#Previously calculated survival limits
survivable = xr.open_dataset('/home/581/cg8052/Survivable_Day-Outdoors_6H-65_over.nc')
Survive1 = survivable['__xarray_dataarray_variable__']

survivable2 = xr.open_dataset('/home/581/cg8052/Survivable_Day-Outdoors_6H-Young_adult.nc')
Survive2 = survivable2['__xarray_dataarray_variable__']

survivable3 = xr.open_dataset('/home/581/cg8052/Survivable_Night-Indoors_6H-65_over.nc')
Survive3 = survivable3['__xarray_dataarray_variable__']

survivable4 = xr.open_dataset('/home/581/cg8052/Survivable_Night-Indoors_6H-Young_adult.nc')
Survive4 = survivable4['__xarray_dataarray_variable__']

In [ ]:
#read in data from desired city
df2 = pd.read_csv(r"D:\ERA5 Data\Seville_2003.csv")
df = pd.DataFrame(df2)
print(df.dtypes)
df = df[12:-60]

In [ ]:
##Make lineplot of time series, changing colour for survivable limit.

fig, ax = plt.subplots(figsize=(10, 7), dpi=500)

# Loop through the data to plot the segments with the correct color
for i in range(len(df) - 3):
    x_values = df['time'].iloc[i:i+2]
    y_values = df['Temp'].iloc[i:i+2]
    RH_value = df['RH'].iloc[i]
    
    if RH_value > df['Indoors_YA'].iloc[i]:
        color = 'magenta'
    elif RH_value > df['Outdoors_YA'].iloc[i]:
        color = 'crimson'
    elif RH_value > df['Indoors_65'].iloc[i]:
        color = 'orange'
    elif RH_value > df['Outdoors_65'].iloc[i]:
        color = 'gold'
    else:
        color = 'cornflowerblue'
    
    ax.plot(x_values, y_values, color=color, linewidth=4)

# Set labels and title
ax.set_xlabel('Time', fontsize=25)
ax.set_ylabel('Temperature ($^\circ$C)', fontsize=25)
#ax.set_title('Temperature Cloncurry, QLD January 2019', fontsize=30)

# Adjust x-ticks: angle them and show every third label
ax.set_xticks(df['time'][::8])
df['time'] = pd.to_datetime(df['time'])
ax.set_xticklabels(df['time'][::8].dt.strftime('%d-%b'), rotation=45, ha='right', fontsize=20)

ax.tick_params(axis='y', labelsize=18)
plt.ylim(28,44)

plt.tight_layout()

# Show the plot
plt.savefig(r"C:\Users\cathe\OneDrive\Pictures\Unsurvivable\Seville_timeseries.jpg", bbox_inches='tight')
plt.show()

In [ ]:
hurs2 = df['RH']
tas2 = df['Temp']

In [ ]:
hurs = hurs2.values.flatten().tolist()
tas = tas2.values.flatten().tolist()

In [ ]:
df =pd.DataFrame({'hurs':hurs, 'tas':tas})


In [ ]:
##Make scatterplots of temp/hum values
fig, ax = plt.subplots(figsize=(20, 12))  # Create a single subplot

# Plot the contour plots on the current axis
Survive1.plot.contour(ax=ax, levels=[0.9, 1.1], colors=['gold', 'white'], label='65+ inside')
Survive2.plot.contour(ax=ax, levels=[0.9, 1.1], colors=['orange', 'white'], label='YA inside')
Survive3.plot.contour(ax=ax, levels=[0.9, 1.1], colors=['crimson', 'white'], label='65+ outside')#, linestyles=['dashed', 'dashed'])
Survive4.plot.contour(ax=ax, levels=[0.9, 1.1], colors=['magenta', 'white'], label='YA outside')#, linestyles=['dashed', 'dashed'])

# Add the additional contour plot with custom formatting
cp = ax.contour(xx, yy, WBT_matrix, colors='Dimgrey', levels=[35], linewidths=(1,))
ax.clabel(cp, fmt='T$_w$ %2.1d°C', colors='Dimgrey', manual=[(40, 70)], fontsize=14)

# Plot the scatter plot on the current axis
plt.scatter(tas, hurs,color='k', s=100)

plt.xticks(fontsize=30)  # Adjust the fontsize as needed
plt.yticks(fontsize=30)

# Set title and labels
ax.set_xlabel('Temperature ($^\circ$C)', fontsize=36)
ax.set_ylabel('Humidity (%)', fontsize=36)
ax.set_title('Temp & RH combinations Cloncurry, QLD Jan 2019', fontsize=40)

plt.tight_layout()
plt.show()